In [2]:
from scipy.stats import wilcoxon
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# import json

In [10]:
all= pd.read_csv('../AllWindowData.csv')
data=pd.DataFrame({'id':all["userId"],"case":all['caseNum'], 'Window Type':all['type'],'Environment': all[ "environment"],'Distance':all["distance"],'Window Anchor':all['windowMode'],'Transparency':all['transparency'],'Size':all['visual angle'],'Meeting Format':all['Meeting type'], 'Theta':all['theta'], 'Phi':all['phi'], 'Pitch':all['pitch'], 'Row':all['row'], 'Offset':all['offset'], 'x': all['x'], 'y': all['y'], 'z': all['z'], 'scale': all['scale']})
data['Window Type'] = data['Window Type'].replace({0: 'Shared Screen', 1: 'Speaker\'s Video'})
data['Window Anchor'] = data['Window Anchor'].replace({0: 'Path Anchor', 1: 'Head Anchor'})
data['Meeting Format'] = data['Meeting Format'].replace({1: 'Listening', 2: 'Discussion'})
data['Environment'] = data['Environment'].replace({'low': 'Low-Traffic', 'high': 'High-Traffic'})
data['Transparency']=1-data['Transparency']

In [12]:
def getStar(p_value):
    significant = ""
    if p_value < 0.001:
        significant = '***'
    elif p_value < 0.01:
        significant = '**'
    elif p_value < 0.05:
        significant = '*'
    return significant
def w(data, para, value):
    print(f'{"Factor":11s}\t{"Window Type":15s}\t\t{"Value":12s}\t{"P-value":7s}\t{"sig":3s}\t{"mean1":7s}\t{"mean2":7s}')
    grouped = data.groupby(['Window Type', para]).apply(lambda x: x[[para, 'Window Type']+value])
    # print(grouped)
    p = grouped[para].unique()
    # All
    a = grouped[(grouped[para]==p[0])]
    b = grouped[(grouped[para]==p[1])]
    for e in value:
        statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
        significant = getStar(p_value)
        print(f'{para:11s}\t{"All":15s}\t\t{e:12s}\t{p_value:.5f}\t{significant}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    #
    for w in ['Shared Screen', 'Speaker\'s Video']:                
        a = grouped[(grouped['Window Type']==w) & (grouped[para]==p[0])]
        b = grouped[(grouped['Window Type']==w) & (grouped[para]==p[1])]
        for e in value:
            statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
            significant = getStar(p_value)
            print(statistic)
            print(f'{para:11s}\t{w:15s}\t\t{e:12s}\t{p_value:.5f}\t{significant:3s}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    print("") 

In [13]:
w(data, 'Environment', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Transparency', 'Distance', 'z'])
w(data, 'Meeting Format', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Transparency', 'Distance', 'z'])
w(data, 'Window Anchor', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Transparency', 'Distance', 'z'])

Factor     	Window Type    		Value       	P-value	sig	mean1  	mean2  
Environment	All            		Size        	0.03410	*	0.28139	0.26723
Environment	All            		Theta       	0.65783		0.81166	-0.44252
Environment	All            		Phi         	0.00017	***	-17.50075	-12.16107
Environment	All            		Pitch       	0.09638		5.82707	4.86351
Environment	All            		Row         	0.92042		-3.54622	-3.18441
Environment	All            		Transparency	0.00000	***	0.86350	0.77872
Environment	All            		Distance    	0.83122		0.86215	0.86634
Environment	All            		z           	0.92818		0.74563	0.74765
436.0
Environment	Shared Screen  		Size        	0.00513	** 	0.37183	0.34084
597.0
Environment	Shared Screen  		Theta       	0.21027	   	-5.87786	-4.71686
384.0
Environment	Shared Screen  		Phi         	0.00202	** 	-18.76713	-12.34873
549.0
Environment	Shared Screen  		Pitch       	0.06407	   	6.91826	4.18784
757.0
Environment	Shared Screen  		Row         	0.91326	   	-0.17186	-

In [6]:
def whp(data, para, value):
    print(f'{"Factor":11s}\t{"Window Anchor"}\t{"Window Type":15s}\t\t{"Value":12s}\t{"P-value":7s}\t{"sig":3s}\t{"mean1":7s}\t{"mean2":7s}')
    grouped = data.groupby(['Window Type', para]).apply(lambda x: x[[para, "Window Anchor", 'Window Type']+value])
    # print(grouped)
    p = grouped[para].unique()
    # All
    for anchor in ['Path Anchor', "Head Anchor"]: 
        a = grouped[(grouped[para]==p[0]) & (grouped['Window Anchor']==anchor)]
        b = grouped[(grouped[para]==p[1]) & (grouped['Window Anchor']==anchor)]
        for e in value:
            statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
            significant = getStar(p_value)
            print(f'{para:11s}\t{anchor:13s}\t{"All":15s}\t\t{e:12s}\t{p_value:.5f}\t{significant}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    #
    for anchor in ['Path Anchor', "Head Anchor"]: 
        for w in ['Shared Screen', 'Speaker\'s Video']:                       
            a = grouped[(grouped['Window Type']==w) & (grouped['Window Anchor']==anchor) & (grouped[para]==p[0])]
            b = grouped[(grouped['Window Type']==w) & (grouped['Window Anchor']==anchor) & (grouped[para]==p[1])]
            for e in value:
                statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
                significant = getStar(p_value)
                print(f'{para:11s}\t{anchor:13s}\t{w:15s}\t\t{e:12s}\t{p_value:.4f}\t{significant:3s}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    print("") 

In [7]:
whp(data, 'Environment', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Transparency', 'Distance', 'z'])
whp(data, 'Meeting Format', ['Size', 'Theta', 'Phi', 'Pitch', 'Row', 'Transparency', 'Distance', 'z'])

Factor     	Window Anchor	Window Type    		Value       	P-value	sig	mean1  	mean2  
Environment	Path Anchor  	All            		Size        	0.16553		0.14863	0.13342
Environment	Path Anchor  	All            		Theta       	0.48808		2.13435	-1.77294
Environment	Path Anchor  	All            		Phi         	0.00160	**	-16.05483	-8.79600
Environment	Path Anchor  	All            		Pitch       	0.07403		5.51962	3.42139
Environment	Path Anchor  	All            		Row         	0.89616		-3.63651	-3.32672
Environment	Path Anchor  	All            		Transparency	0.00272	**	0.87132	0.79579
Environment	Path Anchor  	All            		Distance    	0.26377		0.88760	0.85855
Environment	Path Anchor  	All            		z           	0.40539		0.77007	0.74025
Environment	Head Anchor  	All            		Size        	0.05013		0.14633	0.12153
Environment	Head Anchor  	All            		Theta       	0.07590		-0.51103	1.31157
Environment	Head Anchor  	All            		Phi         	0.23726		-18.94667	-16.46174
Environmen

In [20]:
def wwt(data, value):
    print(f'{"Value":12s}\t{"P-value":7s}\t\t{"sig":3s}\t{"mean1":7s}\t{"mean2":7s}')
    grouped = data.groupby(['Window Type']).apply(lambda x: x[['Window Type']+value])

    # All
    a = grouped[(grouped['Window Type']=='Shared Screen')]
    b = grouped[(grouped['Window Type']=='Speaker\'s Video')]
    for e in value:
        statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
        significant = getStar(p_value)
        print(f'{e:12s}\t{p_value:.5f}\t\t{significant}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    #
    # for w in ['Shared Screen', 'Speaker\'s Video']:                
    #     a = grouped[(grouped['Window Type']==w)]
    #     b = grouped[(grouped['Window Type']==w)]
    #     for e in value:
    #         statistic, p_value = wilcoxon(np.array(a[e].tolist()), np.array(b[e].tolist()), alternative='two-sided')
    #         significant = getStar(p_value)
    #         print(f'{w:15s}\t\t{e:12s}\t{p_value:.5f}\t{significant:3s}\t{a[e].mean():.5f}\t{b[e].mean():.5f}')
    print("") 

In [21]:
wwt(data, ['Size', 'Offset', 'Theta', 'Phi', 'Transparency'])

Value       	P-value		sig	mean1  	mean2  
Size        	0.00000		***	0.35421	0.19240
Offset      	0.00005		***	2.50064	2.37038
Theta       	0.00001		***	1.65888	1.47257
Phi         	0.07924			1.85226	1.81519
Transparency	0.49844			0.82223	0.81770

